<a href="https://colab.research.google.com/github/avighna-tripathi/JOEY-BOT/blob/main/JOEY_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import random

dialogues = []

def strip_parentheses(s):
    return re.sub(r'\(.*?\)', '', s)

def is_character_line(line, character_name):
    # Use a case-insensitive regex pattern to match the character name followed by a colon
    pattern = re.compile(rf'^{character_name}:\s', re.IGNORECASE)
    return bool(pattern.match(line))

def extract_character_lines(file_path, character_name):
    lines = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as script_file:
        try:
            lines = script_file.readlines()
        except UnicodeDecodeError:
            pass

    current_character = ''
    for line in lines:
        stripped_line = line.strip()
        if is_character_line(stripped_line, character_name):
            current_character = character_name
            dialog_line = strip_parentheses(stripped_line.split(':', 1)[1]).strip()
            if len(dialog_line) > 0:
                dialogues.append(dialog_line)
        elif current_character.lower() == character_name.lower() and stripped_line:
            dialog_line = strip_parentheses(stripped_line).strip()
            if len(dialog_line) > 0:
                dialogues.append(dialog_line)
        else:
            current_character = ''

def process_directory(directory_path, character_name):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):  # Ignore directories
            extract_character_lines(file_path, character_name)


In [3]:
process_directory("./sample_data/friends",'Joey')

In [4]:
print(len(dialogues))

814


In [5]:
!pip install openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00


In [6]:
!pip install ragas langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [7]:
import openai
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

openai.api_key=api_key

In [8]:
with open("./sample_data/joey_lines.txt","w+") as f:
  for line in dialogues:
    f.write(line + "\n")

In [9]:
#Source: sample code from langchain docs
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

In [10]:
!pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 5.1 MB/s eta 0:00:00


In [11]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(OpenAIEmbeddings(openai_api_key=api_key),breakpoint_threshold_type="percentile")
with open("./sample_data/joey_lines.txt") as f:
  joey_lines = f.read()
docs = text_splitter.create_documents([joey_lines])

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
index = VectorstoreIndexCreator(embedding=embeddings).from_documents(docs)


/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers import RePhraseQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

llm=ChatOpenAI(openai_api_key=api_key,temperature=0)
system_prompt=(
    "You are Joey from F.R.I.E.N.D.S ."
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)
retriever=index.vectorstore.as_retriever(search_kwargs={'k': 5})

In [13]:
retriever_from_llm=RePhraseQueryRetriever.from_llm(
    retriever=retriever,llm=llm
)

In [14]:
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever_from_llm
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(compression_retriever, question_answer_chain)

In [15]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

compressed_docs = compression_retriever.invoke("Who is the sister of Ross ?")
pretty_print_docs(compressed_docs)

Document 1:

. gay. Woah, woah, woah, you have a date? But uh, uh, what about uh, Ross and uh. .
----------------------------------------------------------------------------------------------------
Document 2:

What? Come on, they're close. Oh, like you've never gotten a little rambunctious with Ross. Well, who's to say what's true?
----------------------------------------------------------------------------------------------------
Document 3:

Oh, yeah, I do. [quietly] I never know how long you're supposed to wait in this type of a situation before you can talk again, you know? [Ross stares blankly at him] Maybe a little longer. Ahhhhhh, I didn't get the job. I dunno. Some fat guy's sleeping with the store manager. He's not even jolly, it's all political. Ah, I'm gonna be one of his helpers. It's just such a slap in the face, y'know? Hey, that guy's going home with more than a note! Hi. Hi, sorry I'm late. Nice shoes, huh? You know more than one Fun Bobby? Ooh ooh ooh ooh, there's no 

In [16]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.re_phraser").setLevel(logging.INFO)


question = input("Hey !! how you doin' ? Ask me anything related to season 1 :")

result = chain.invoke({"input": question})
print("SOURCE DOCUMENTS:\n")
for doc in result["context"]:
   print(doc)
print("\nRESULT:\n")
print(result["answer"])
#s=input("Wanna ask me more ques [y/n] ?:")
  # if s=='y':
  #   continue
  # else :
  #   break

Hey !! how you doin' ? Ask me anything related to season 1 :Who is the mother of Chandler ?


INFO:langchain.retrievers.re_phraser:Re-phrased question: Query for vectorstore: Mother of Chandler


SOURCE DOCUMENTS:

page_content='Yeah. We did? Uh, hi. We uh, we used to work together. Yeah, at Macy's. You were the Obsession girl, right? I was the Aramis guy. Aramis? Aramis? I gotta tell you. You're the best in the business. I'm serious. You're amazing. You know when to spritz, when to lay back. Ooh, you smell great tonight. What're you wearing? Listen, uh, you wanna go get a drink or something? What's wrong? Oh. What? Wait, wait, wait! So I guess you all saw it. Set another place for Thanksgiving. My entire family thinks I have VD. Chandler, will you just come in already? Hey, Monica, I got a question. I don't see any tater tots. But my mom always makes them. It's like a tradition. You get a little piece of turkey on your fork, a little cranberry sauce, and a tot! It's bad enough I can't be with my family because of my disease. The balloon? Wait, wait, we have a copy of your key. That tone will not make me go any faster. That one will.'
page_content='what else? Alright, look, I'l

In [17]:
eval_questions=[
    "Where does chandler work ?",
    "Who is Chandler's best friend ?"
    "Who is Ross ?",
    "Where does Monica work ?",
    "What is the real name of Pheebs ?"
]

eval_answers=[
    "Chandler works at a consultancy firm",
    "Ross is Chandler's best friend",
    "Ross is a character in the TV serial F.R.I.E.N.D.S.",
    "She is a chef in Central perk",
    "Real name of Pheebs is Phoebe Buffay"
]

In [18]:
result = chain.invoke({"input": eval_questions[1]})
print(result)

INFO:langchain.retrievers.re_phraser:Re-phrased question: Query for vectorstore: Chandler's best friend


{'input': "Who is Chandler's best friend ?Who is Ross ?", 'context': [_DocumentWithState(page_content="So, tell me. Was it like you and Chandler, and then you and me, or you and me and Chandler? What? Listen, the next time you talk to him, can you ask him which one the strongest Power Ranger is? Stegosaurus! What about Andr ? They're here already? Uh, moist, yeah. All right, all right. For the ride. All right, I'm going. I'm going. Um, this? What up?! Hello? Oh, hi. Yeah, hold on a second. Ross, it's Julie, for you. Needs some clothes altered? Why don't you go see Frankie? My family's been goin' to him forever. He did my first suit when I was 15. No wait, 16. No, 'scuse me, 15. All right, when was 1990? You know, I think I was sixteen. No-no-no-no-no,  I'm not fallin' for that again. Whoa, whoa. And the fact that you dumped him because he hit on Phoebe? Hey, Chandler, when you see Frankie, tell him Joey Tribbiani says hello. He'll know what it means. Can we see her? How're you doing? O

In [19]:
answers=[]
contexts=[]
for questions in eval_questions:
  response = chain.invoke({"input": questions})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

INFO:langchain.retrievers.re_phraser:Re-phrased question: The converted query for the vectorstore would be: "Chandler work location"
INFO:langchain.retrievers.re_phraser:Re-phrased question: Query for vectorstore: Chandler's best friend
INFO:langchain.retrievers.re_phraser:Re-phrased question: Query for vectorstore: Monica work location
INFO:langchain.retrievers.re_phraser:Re-phrased question: Query for vectorstore: Real name of Pheebs


In [26]:
# Check lengths of all lists
from datasets import Dataset
len_eval_questions = len(eval_questions)
len_answers = len(answers)
len_contexts = len(contexts)
len_ground_truth = len(eval_answers)

print(f"Length of eval_questions: {len_eval_questions}")
print(f"Length of answers: {len_answers}")
print(f"Length of contexts: {len_contexts}")
print(f"Length of ground_truth: {len_ground_truth}")

# Ensure all lists have the same length
min_length = min(len_eval_questions, len_answers, len_contexts, len_ground_truth)

# Trim lists to the same length if necessary
eval_questions = eval_questions[:min_length]
answers = answers[:min_length]
contexts = contexts[:min_length]
eval_answers = eval_answers[:min_length]

# Create Dataset
response_datasets = Dataset.from_dict({
    "question" : eval_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth" : eval_answers
})

# Check the first record
print(response_datasets[0])

Length of eval_questions: 4
Length of answers: 4
Length of contexts: 4
Length of ground_truth: 4
{'question': 'Where does chandler work ?', 'answer': 'Chandler works in advertising as a statistical analysis and data reconfiguration.', 'contexts': ["what else? Alright, look, I'll start, OK? Come on. Come on. Alright, ready, look! Oh... Ross.... you get me so hot. I want your lips on me now. Alright, now you say something. Come on! You like this woman, right? You want to see her again, right? Well if you can't talk dirty to me, how're you going to talk dirty to her? Now tell me you want to caress my butt! Alright, alright. I'm around. Go ahead. There you go! Keep going. Keep going! Say it... say it! You're not going to believe this! Hey Chandler, while you were sleeping that guy from your old job called again. And again, and again, and again... Hello? And again. What a tool! So, er... how did it go with Celia? All right, Ross! Whoa! And the... huh-huh? You cuddled. Yeah, where the hell i

In [31]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [32]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [33]:
import os
os.environ['OPENAI_API_KEY'] = api_key
results = evaluate(response_datasets, metrics)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [34]:
results

{'faithfulness': 0.0000, 'answer_relevancy': 0.9332, 'context_recall': 0.0000, 'context_precision': 0.2500, 'answer_correctness': 0.3063}

In [35]:
results.to_pandas()

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,Where does chandler work ?,Chandler works in advertising as a statistical...,"[what else? Alright, look, I'll start, OK? Com...",Chandler works at a consultancy firm,0.0,0.939744,0.0,0.0,0.230132
1,Who is Chandler's best friend ?Who is Ross ?,"Chandler's best friend is Joey, and Ross is Mo...","[So, tell me. Was it like you and Chandler, an...",Ross is Chandler's best friend,0.0,0.930543,0.0,1.0,0.605064
2,Where does Monica work ?,Monica works as a chef at a restaurant called ...,"[Yeah. We did? Uh, hi. We uh, we used to work ...",Ross is a character in the TV serial F.R.I.E.N...,0.0,0.986448,0.0,0.0,0.189568
3,What is the real name of Pheebs ?,Her real name is Phoebe Buffay.,"[Yeah. Hey Pheebs, guess who we saw today. Nop...",She is a chef in Central perk,0.0,0.876258,0.0,0.0,0.200536
